In [1]:
!pip -q install rank_bm25
!pip -q install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 74.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.2 MB/s eta 0:00:00


In [2]:
import os
import json
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
from pyvi.ViTokenizer import tokenize

In [10]:
# Đường dẫn tới thư mục chứa các file JSON
qa_folder_path = '/kaggle/input/all-dataset-qa-retrieve/all_dataset_qa_retrieve'

# Danh sách để chứa dữ liệu
qa_data = []

# Lặp qua tất cả các file trong thư mục và sử dụng tqdm để theo dõi tiến trình
for file_name in tqdm(os.listdir(qa_folder_path), desc="Reading files"):
    
    if file_name.endswith('.json'):  # Kiểm tra nếu file là file JSON
        file_path = os.path.join(qa_folder_path, file_name)
        
        # Mở và đọc dữ liệu từ file JSON
        with open(file_path, 'r', encoding='utf-8') as f:
            qa_file_data = json.load(f)
            
            # Lặp qua các câu hỏi và câu trả lời trong dữ liệu
            for qa in qa_file_data['qa']:
                qa_data.append({
                    'question': qa['question'],
                    'answer': qa['answer'],
                    'id': qa['id']
                })

Reading files: 100%|██████████| 499/499 [00:01<00:00, 396.73it/s]


In [11]:
# Tạo DataFrame từ dữ liệu đã thu thập
qa_df = pd.DataFrame(qa_data)

In [13]:
qa_df['tokenized_question'] = qa_df['question'].apply(lambda x: tokenize(x.lower()).split())

In [14]:
import pickle

# Load lại bm25 model
with open('/kaggle/input/bm25pyvi/other/default/1/bm25_model_pyvi.pkl', 'rb') as f:
    bm25 = pickle.load(f)


In [15]:
chunked_df = pd.read_json('/kaggle/input/df-full-and-chunked/df/chunked_df.json')

In [17]:
!pip install -q sentence-transformers
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.9 MB/s eta 0:00:00


config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
from tqdm import tqdm
import numpy as np
from sentence_transformers import CrossEncoder

def evaluate_bm25_with_crossencoder_top_k(df, qa_df, bm25, top_k_bm25=200, top_k_values=list(range(1, 101))):
    results = {f"Top-{k}": 0 for k in top_k_values}  # Khởi tạo kết quả đánh giá
    total_queries = len(qa_df)

    for _, row in tqdm(qa_df.iterrows(), total=total_queries):
        query = row['tokenized_question']
        true_id = row['id']

        # Step 1: Dùng BM25 lấy Top-200
        scores = bm25.get_scores(query)
        top_indices = scores.argsort()[::-1][:top_k_bm25]  # Top-200 kết quả từ BM25
        top_candidates = df.iloc[top_indices]

        # Step 2: Tạo input cho CrossEncoder
        crossencoder_inputs = [(row['question'], chunk) for chunk in top_candidates['content']]

        # Step 3: Tính điểm bằng CrossEncoder
        crossencoder_scores = model.predict(crossencoder_inputs)

        # Step 4: Chọn Top-100 từ CrossEncoder
        cross_top_indices = np.argsort(crossencoder_scores)[::-1][:len(top_k_values)]
        top_ids = top_candidates.iloc[cross_top_indices]['id'].tolist()

        # Step 5: Đánh giá cho từng Top-k
        for k in top_k_values:
            if true_id in top_ids[:k]:
                results[f"Top-{k}"] += 1

    # Tính độ chính xác cho mỗi Top-k
    for k in top_k_values:
        results[f"Top-{k}"] /= total_queries

    return results

# Chạy đánh giá
top_k_values = list(range(1, 101))  # Top-1 đến Top-100
evaluation_results = evaluate_bm25_with_crossencoder_top_k(chunked_df, qa_df, bm25, top_k_bm25=200, top_k_values=top_k_values)

# In kết quả đánh giá
for k, v in evaluation_results.items():
    print(f"{k}: {v * 100:.2f}%")

  0%|          | 0/4121 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 1/4121 [00:04<5:09:45,  4.51s/it]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 2/4121 [00:08<4:31:26,  3.95s/it]

...

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

100%|█████████▉| 4120/4121 [3:44:35<00:02,  2.90s/it]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 4121/4121 [3:44:38<00:00,  3.27s/it]

Top-1: 48.53%
Top-2: 59.74%
Top-3: 65.86%
Top-4: 69.47%
Top-5: 71.85%
Top-6: 73.74%
Top-7: 75.49%
Top-8: 76.56%
Top-9: 77.60%
Top-10: 78.62%
Top-11: 79.57%
Top-12: 80.42%
Top-13: 81.19%
Top-14: 81.78%
Top-15: 82.48%
Top-16: 82.92%
Top-17: 83.33%
Top-18: 83.57%
Top-19: 83.98%
Top-20: 84.42%
Top-21: 84.86%
Top-22: 85.20%
Top-23: 85.59%
Top-24: 85.88%
Top-25: 86.19%
Top-26: 86.44%
Top-27: 86.58%
Top-28: 86.87%
Top-29: 87.02%
Top-30: 87.21%
Top-31: 87.41%
Top-32: 87.58%
Top-33: 87.75%
Top-34: 87.96%
Top-35: 88.16%
Top-36: 88.33%
Top-37: 88.57%
Top-38: 88.67%
Top-39: 88.84%
Top-40: 88.98%
Top-41: 89.10%
Top-42: 89.15%
Top-43: 89.37%
Top-44: 89.52%
Top-45: 89.64%
Top-46: 89.81%
Top-47: 89.86%
Top-48: 89.93%
Top-49: 89.98%
Top-50: 90.05%
Top-51: 90.10%
Top-52: 90.17%
Top-53: 90.22%
Top-54: 90.27%
Top-55: 90.39%
Top-56: 90.44%
Top-57: 90.51%
Top-58: 90.58%
Top-59: 90.75%
Top-60: 90.88%
Top-61: 90.95%
Top-62: 90.97%
Top-63: 91.00%
Top-64: 91.17%
Top-65: 91.22%
Top-66: 91.26%
Top-67: 91.31%
Top-

In [19]:
for k, v in evaluation_results.items():
    print(f"{k}: {v * 100:.2f}%")

Top-1: 48.53%
Top-2: 59.74%
Top-3: 65.86%
Top-4: 69.47%
Top-5: 71.85%
Top-6: 73.74%
Top-7: 75.49%
Top-8: 76.56%
Top-9: 77.60%
Top-10: 78.62%
Top-11: 79.57%
Top-12: 80.42%
Top-13: 81.19%
Top-14: 81.78%
Top-15: 82.48%
Top-16: 82.92%
Top-17: 83.33%
Top-18: 83.57%
Top-19: 83.98%
Top-20: 84.42%
Top-21: 84.86%
Top-22: 85.20%
Top-23: 85.59%
Top-24: 85.88%
Top-25: 86.19%
Top-26: 86.44%
Top-27: 86.58%
Top-28: 86.87%
Top-29: 87.02%
Top-30: 87.21%
Top-31: 87.41%
Top-32: 87.58%
Top-33: 87.75%
Top-34: 87.96%
Top-35: 88.16%
Top-36: 88.33%
Top-37: 88.57%
Top-38: 88.67%
Top-39: 88.84%
Top-40: 88.98%
Top-41: 89.10%
Top-42: 89.15%
Top-43: 89.37%
Top-44: 89.52%
Top-45: 89.64%
Top-46: 89.81%
Top-47: 89.86%
Top-48: 89.93%
Top-49: 89.98%
Top-50: 90.05%
Top-51: 90.10%
Top-52: 90.17%
Top-53: 90.22%
Top-54: 90.27%
Top-55: 90.39%
Top-56: 90.44%
Top-57: 90.51%
Top-58: 90.58%
Top-59: 90.75%
Top-60: 90.88%
Top-61: 90.95%
Top-62: 90.97%
Top-63: 91.00%
Top-64: 91.17%
Top-65: 91.22%
Top-66: 91.26%
Top-67: 91.31%
Top-